In [38]:
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
import random
import face_recognition
import time

In [39]:
DETECT_CONF_THRESH = 0.5
RECOG_CONF_THRESH = 0.5
image_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/datasets/caltech_faces_test/"
detector_main_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/face_detection_model/"
encodings_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_encodings.pickle"
recognizer_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_recognizer.pickle"
label_encoder_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_label_encoder.pickle"


In [40]:
# Load face detector model
print("[INFO] loading face detector...")
protoPath = os.path.join(detector_main_folder, "deploy.prototxt")
modelPath = os.path.join(detector_main_folder,"res10_300x300_ssd_iter_140000.caffemodel")
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
print("[INFO] face detector loaded.")
# load the actual face recognition model along with the label encoder
print("[INFO] loading trained face recognizer model...")
recognizer = pickle.loads(open(recognizer_save_path, "rb").read())
le = pickle.loads(open(label_encoder_save_path, "rb").read())



[INFO] loading face detector...
[INFO] face detector loaded.
[INFO] loading trained face recognizer model...


In [41]:
# load the image, resize it to have a width of 600 pixels (while
# maintaining the aspect ratio), and then grab the image dimensions
all_images = os.listdir(image_folder)
print("Total images in the specified folder: ", len(all_images))
for i in range(10):
    time_1 = time.time()
    random_image = random.choice(all_images)
    true_name = random_image.split()[0]
    image_path = os.path.join(image_folder, random_image)
    image = cv2.imread(image_path)

    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    
    # apply OpenCV's deep learning-based face detector to localize faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections
        if confidence > DETECT_CONF_THRESH:
            # compute the (x, y)-coordinates of the bounding box for the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Need to convert because the face_recognition.face_encoding() doesn't use coordinate system
            box = [(startY, endX, endY, startX)]
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Compute encoding using face_recognition library
            encodings = np.array(face_recognition.face_encodings(rgb, box))

            # perform classification to recognize the face
            preds = recognizer.predict_proba(encodings)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]

            # draw the bounding box of the face along with the associated probability
            text_1 = "True: {} Predicted: {}: {:.2f}%".format(true_name, name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            if( true_name == name):
                color = (0, 255, 0)
            else:
                color = (0, 0, 255)
            cv2.putText(image, text_1, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    # show the output image
    cv2.imshow("Image", image)
    time_2 = time.time()
    print(f"time taken to recognize {random_image} is {time_2-time_1} seconds")
    print(f"True Label: {true_name}, Predicted Label: {name}")
    key = cv2.waitKey(0)
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue
cv2.destroyAllWindows()

Total images in the specified folder:  49
time taken to recognize person_d (21).jpg is 0.6276602745056152 seconds
True Label: person_d, Predicted Label: person_d
time taken to recognize person_l (14).jpg is 0.821460485458374 seconds
True Label: person_l, Predicted Label: shubhranshu_malhotra
time taken to recognize person_m (18).jpg is 0.44600367546081543 seconds
True Label: person_m, Predicted Label: person_m
time taken to recognize shubhranshu_malhotra (2).jpg is 0.45745420455932617 seconds
True Label: shubhranshu_malhotra, Predicted Label: shubhranshu_malhotra
time taken to recognize person_t (22).jpg is 0.4581930637359619 seconds
True Label: person_t, Predicted Label: person_t
time taken to recognize person_f (20).jpg is 0.42981529235839844 seconds
True Label: person_f, Predicted Label: person_f
time taken to recognize person_q (19).jpg is 0.46090221405029297 seconds
True Label: person_q, Predicted Label: person_q
time taken to recognize person_l (22).jpg is 0.4319887161254883 seco